# Partie 2 : Transformation des Données

Ce notebook présente le processus de transformation des données brutes pour notre projet ETL sur les pandémies. Nous allons nettoyer, agréger, normaliser et supprimer les doublons des données extraites dans la phase précédente.

## Importation des bibliothèques nécessaires

In [ ]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration pour afficher plus de colonnes
pd.set_option('display.max_columns', None)

## Chargement des données brutes

In [ ]:
# Répertoire des données brutes
raw_data_dir = './Donnees Brutes/'

# Chemins des fichiers de données
covid_data_path = os.path.join(raw_data_dir, 'covid_19_clean_complete.csv')
monkeypox_data_path = os.path.join(raw_data_dir, 'owid-monkeypox-data.csv')
worldometer_data_path = os.path.join(raw_data_dir, 'worldometer_coronavirus_daily_data.csv')

# Chargement des données
try:
    covid_df = pd.read_csv(covid_data_path)
    print(f"Données COVID-19 chargées avec succès. Forme: {covid_df.shape}")
except Exception as e:
    print(f"Erreur lors du chargement des données COVID-19: {e}")
    covid_df = None

try:
    monkeypox_df = pd.read_csv(monkeypox_data_path)
    print(f"Données Monkeypox chargées avec succès. Forme: {monkeypox_df.shape}")
except Exception as e:
    print(f"Erreur lors du chargement des données Monkeypox: {e}")
    monkeypox_df = None
    
try:
    worldometer_df = pd.read_csv(worldometer_data_path)
    print(f"Données Worldometer chargées avec succès. Forme: {worldometer_df.shape}")
except Exception as e:
    print(f"Erreur lors du chargement des données Worldometer: {e}")
    worldometer_df = None

## Fonctions de nettoyage et transformation

In [ ]:
def clean_covid_data(df):
    """
    Fonction pour nettoyer les données COVID-19
    """
    if df is None:
        return None
    
    # Copie du DataFrame pour éviter les modifications en place
    cleaned_df = df.copy()
    
    # Conversion des dates en format datetime
    if 'Date' in cleaned_df.columns:
        cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'])
    elif 'date' in cleaned_df.columns:
        cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])
    
    # Normalisation des noms de colonnes
    cleaned_df.columns = [col.lower().replace(' ', '_') for col in cleaned_df.columns]
    
    # Gestion des valeurs manquantes
    for col in cleaned_df.select_dtypes(include=['float64', 'int64']).columns:
        cleaned_df[col] = cleaned_df[col].fillna(0)
    
    # Normalisation des noms de pays
    if 'country' in cleaned_df.columns:
        # Exemple de normalisation de quelques noms de pays
        country_mapping = {
            'US': 'États-Unis',
            'United States': 'États-Unis',
            'USA': 'États-Unis',
            'UK': 'Royaume-Uni',
            'United Kingdom': 'Royaume-Uni',
            'Mainland China': 'Chine',
            'Korea, South': 'Corée du Sud',
            'South Korea': 'Corée du Sud',
            'Taiwan*': 'Taiwan',
            'Iran (Islamic Republic of)': 'Iran',
            'Hong Kong SAR': 'Hong Kong'
        }
        cleaned_df['country'] = cleaned_df['country'].replace(country_mapping)
    
    # Suppression des doublons
    cleaned_df = cleaned_df.drop_duplicates()
    
    print(f"Nettoyage des données COVID-19 terminé. Nouvelle forme: {cleaned_df.shape}")
    return cleaned_df

In [ ]:
def clean_monkeypox_data(df):
    """
    Fonction pour nettoyer les données Monkeypox
    """
    if df is None:
        return None
    
    # Copie du DataFrame pour éviter les modifications en place
    cleaned_df = df.copy()
    
    # Conversion des dates en format datetime
    if 'Date' in cleaned_df.columns:
        cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'])
    elif 'date' in cleaned_df.columns:
        cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])
    
    # Normalisation des noms de colonnes
    cleaned_df.columns = [col.lower().replace(' ', '_') for col in cleaned_df.columns]
    
    # Gestion des valeurs manquantes
    for col in cleaned_df.select_dtypes(include=['float64', 'int64']).columns:
        cleaned_df[col] = cleaned_df[col].fillna(0)
    
    # Normalisation des noms de pays
    if 'entity' in cleaned_df.columns:
        # Renommer la colonne entity en country pour uniformité
        cleaned_df = cleaned_df.rename(columns={'entity': 'country'})
        
        # Exemple de normalisation de quelques noms de pays
        country_mapping = {
            'United States': 'États-Unis',
            'USA': 'États-Unis',
            'United Kingdom': 'Royaume-Uni',
            'South Korea': 'Corée du Sud'
        }
        cleaned_df['country'] = cleaned_df['country'].replace(country_mapping)
    
    # Suppression des doublons
    cleaned_df = cleaned_df.drop_duplicates()
    
    print(f"Nettoyage des données Monkeypox terminé. Nouvelle forme: {cleaned_df.shape}")
    return cleaned_df

In [ ]:
def clean_worldometer_data(df):
    """
    Fonction pour nettoyer les données Worldometer
    """
    if df is None:
        return None
    
    # Copie du DataFrame pour éviter les modifications en place
    cleaned_df = df.copy()
    
    # Conversion des dates en format datetime
    if 'Date' in cleaned_df.columns:
        cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'])
    elif 'date' in cleaned_df.columns:
        cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])
    
    # Normalisation des noms de colonnes
    cleaned_df.columns = [col.lower().replace(' ', '_') for col in cleaned_df.columns]
    
    # Gestion des valeurs manquantes
    for col in cleaned_df.select_dtypes(include=['float64', 'int64']).columns:
        cleaned_df[col] = cleaned_df[col].fillna(0)
    
    # Normalisation des noms de pays
    if 'country' in cleaned_df.columns:
        # Exemple de normalisation de quelques noms de pays
        country_mapping = {
            'US': 'États-Unis',
            'USA': 'États-Unis',
            'UK': 'Royaume-Uni',
            'S. Korea': 'Corée du Sud',
            'UAE': 'Émirats Arabes Unis'
        }
        cleaned_df['country'] = cleaned_df['country'].replace(country_mapping)
    
    # Suppression des doublons
    cleaned_df = cleaned_df.drop_duplicates()
    
    print(f"Nettoyage des données Worldometer terminé. Nouvelle forme: {cleaned_df.shape}")
    return cleaned_df

## Application des fonctions de nettoyage

In [ ]:
# Nettoyage des données COVID-19
covid_clean_df = clean_covid_data(covid_df)

# Affichage des premières lignes
if covid_clean_df is not None:
    display(covid_clean_df.head())
    print("\nInformations sur les colonnes:")
    display(covid_clean_df.info())

In [ ]:
# Nettoyage des données Monkeypox
monkeypox_clean_df = clean_monkeypox_data(monkeypox_df)

# Affichage des premières lignes
if monkeypox_clean_df is not None:
    display(monkeypox_clean_df.head())
    print("\nInformations sur les colonnes:")
    display(monkeypox_clean_df.info())

In [ ]:
# Nettoyage des données Worldometer
worldometer_clean_df = clean_worldometer_data(worldometer_df)

# Affichage des premières lignes
if worldometer_clean_df is not None:
    display(worldometer_clean_df.head())
    print("\nInformations sur les colonnes:")
    display(worldometer_clean_df.info())

## Transformation pour la préparation des données

In [ ]:
def prepare_confirmed_cases(covid_df, worldometer_df):
    """
    Fonction pour préparer les données de cas confirmés
    """
    # Initialisation d'un DataFrame vide
    confirmed_df = pd.DataFrame()
    
    # Traitement des données COVID-19
    if covid_df is not None and 'confirmed' in covid_df.columns:
        # Sélection des colonnes pertinentes
        covid_confirmed = covid_df[['country', 'date', 'confirmed']].copy()
        
        # Calcul des nouveaux cas quotidiens
        covid_confirmed['nouveaux_cas'] = covid_confirmed.groupby('country')['confirmed'].diff().fillna(0).astype(int)
        
        # Renommage des colonnes
        covid_confirmed = covid_confirmed.rename(columns={'confirmed': 'cas_cumules'})
        
        # Ajout d'une colonne pour identifier la pandémie
        covid_confirmed['pandemie'] = 'COVID-19'
        
        # Ajout au DataFrame principal
        confirmed_df = pd.concat([confirmed_df, covid_confirmed])
    
    # Traitement des données Worldometer
    if worldometer_df is not None and 'total_cases' in worldometer_df.columns:
        # Sélection des colonnes pertinentes
        worldometer_confirmed = worldometer_df[['country', 'date', 'total_cases', 'new_cases']].copy()
        
        # Renommage des colonnes
        worldometer_confirmed = worldometer_confirmed.rename(columns={
            'total_cases': 'cas_cumules',
            'new_cases': 'nouveaux_cas'
        })
        
        # Ajout d'une colonne pour identifier la pandémie
        worldometer_confirmed['pandemie'] = 'COVID-19'
        
        # Ajout au DataFrame principal
        confirmed_df = pd.concat([confirmed_df, worldometer_confirmed])
    
    # Suppression des doublons potentiels
    confirmed_df = confirmed_df.drop_duplicates(subset=['country', 'date'])
    
    # Tri par pays et date
    confirmed_df = confirmed_df.sort_values(['country', 'date'])
    
    print(f"Préparation des données de cas confirmés terminée. Forme: {confirmed_df.shape}")
    return confirmed_df

In [ ]:
def prepare_deaths(covid_df, worldometer_df):
    """
    Fonction pour préparer les données de décès
    """
    # Initialisation d'un DataFrame vide
    deaths_df = pd.DataFrame()
    
    # Traitement des données COVID-19
    if covid_df is not None and 'deaths' in covid_df.columns:
        # Sélection des colonnes pertinentes
        covid_deaths = covid_df[['country', 'date', 'deaths']].copy()
        
        # Calcul des nouveaux décès quotidiens
        covid_deaths['nouveaux_deces'] = covid_deaths.groupby('country')['deaths'].diff().fillna(0).astype(int)
        
        # Renommage des colonnes
        covid_deaths = covid_deaths.rename(columns={'deaths': 'deces_cumules'})
        
        # Ajout d'une colonne pour identifier la pandémie
        covid_deaths['pandemie'] = 'COVID-19'
        
        # Ajout au DataFrame principal
        deaths_df = pd.concat([deaths_df, covid_deaths])
    
    # Traitement des données Worldometer
    if worldometer_df is not None and 'total_deaths' in worldometer_df.columns:
        # Sélection des colonnes pertinentes
        worldometer_deaths = worldometer_df[['country', 'date', 'total_deaths', 'new_deaths']].copy()
        
        # Renommage des colonnes
        worldometer_deaths = worldometer_deaths.rename(columns={
            'total_deaths': 'deces_cumules',
            'new_deaths': 'nouveaux_deces'
        })
        
        # Ajout d'une colonne pour identifier la pandémie
        worldometer_deaths['pandemie'] = 'COVID-19'
        
        # Ajout au DataFrame principal
        deaths_df = pd.concat([deaths_df, worldometer_deaths])
    
    # Suppression des doublons potentiels
    deaths_df = deaths_df.drop_duplicates(subset=['country', 'date'])
    
    # Tri par pays et date
    deaths_df = deaths_df.sort_values(['country', 'date'])
    
    print(f"Préparation des données de décès terminée. Forme: {deaths_df.shape}")
    return deaths_df